Election de 2022 premier tour

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import os
os.listdir()

#### Importer la database

In [ ]:
from db import SessionLocal
from models import Departement

session = SessionLocal()



Import des datasets.

In [ ]:
df = pd.read_excel('./elections/presidentielles-2022-1.xlsx')

Visualise toutes les colones

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head()

In [ ]:
df.shape

Recuperer les information généraux sur les élections départemental

In [ ]:
df_infos_general = df.iloc[:, :17]
df_infos_general.head()

Renommer les colonnes des information généraux sur les élections départemental

In [ ]:
df_infos_general.columns = [
    "code_dept", "nom_dept", "etat_saisie", "nb_inscrits", "nb_abstentions",
    "pct_abstentions", "nb_votants", "pct_votants", "nb_blancs",
    "pct_blancs_inscrits", "pct_blancs_votants", "nb_nuls",
    "pct_nuls_inscrits", "pct_nuls_votants", "nb_exprimes",
    "pct_exprimes_inscrits", "pct_exprimes_votants"
]
df_infos_general.head()

## Analyse exploratoire des données

Elle sert à vérifier que les données sont complètes et cohérentes, à identifier ce qui est réellement informatif (variables utiles) et ce qui est redondant ou inutile, à détecter d’éventuelles anomalies ou valeurs aberrantes, et à appréhender la structure globale des données (volumes, proportions, variabilité entre départements).

#### Structure globale du DataFrame

Permet de repérer les colonnes vides, les types incorrects, et d’avoir une première idée de la taille du jeu de données.

In [ ]:
df_infos_general.info()

#### Analyser les valeurs manquantes

Analyser les valeurs manquantes est essentiel car leur présence peut fausser l’analyse statistique et les modèles de prédiction : une valeur absente peut modifier les moyennes, les pourcentages ou les corrélations, introduire des biais et provoquer des erreurs ou de mauvaises performances lors de l’entraînement des modèles de machine learning.

In [ ]:
df_infos_general.isnull().sum()

#### Analyse descriptive statistique sur les colonnes numériques

Permet de comprendre la distribution, détecter outliers, différences de taille entre départements, etc.

Colonnes numériques pour analyse

In [ ]:
colonnes_numeriques = df_infos_general.select_dtypes(include=["int64", "float64"]).columns
df_num = df_infos_general[colonnes_numeriques]

print(df_num.describe())

In [ ]:
# 5a. Catégorielles restantes (ici code_dept)
for col in df_infos_general.select_dtypes(include=["object"]).columns:
    counts = df_infos_general[col].value_counts()
    sns.countplot(data=df_infos_general, x=col, order=counts.index)
    plt.title(f"Distribution de '{col}'")
    plt.xticks(rotation=45)
    plt.show()

#### Analyse descriptive des variables 

Permet de détecter les colonnes constantes ou redondantes

In [ ]:
df_infos_general.describe(include=["object"])

In [ ]:
# 5b. Numériques
for col in df_num.columns:
    sns.histplot(df_num[col], bins=10, kde=True)
    plt.title(f"Distribution de '{col}'")
    plt.show()

Recuperer que les ligne qui ont un état Complet

In [ ]:
df_infos_general = df_infos_general[df_infos_general["etat_saisie"] == "Complet"]

In [ ]:
msno.matrix(df)


Visualiser les valeurs manquantes

#### Analyser la distribution

Analyser la distribution des données permet de comprendre comment les valeurs se répartissent (concentration, dispersion, asymétrie) et de détecter d’éventuelles anomalies comme des valeurs aberrantes ou incohérentes.

Analyser les valeurs aberrantes

Analyse les boxes splots

In [ ]:
# 6️⃣ Boxplots pour détection des valeurs aberrantes
for col in df_num.columns:
    sns.boxplot(x=df_num[col])
    plt.title(f"Boxplot de '{col}'")
    plt.show()


Matrice de corrélation pour metre en evidance la muticolinéarité entre les variables numériques et un VIF pour mesure à quel point la variance d’un coefficient de régression est augmentée parce qu’une variable explicative est corrélée aux autres.

In [ ]:
# =========================
# 1️⃣ Matrice de corrélation
# =========================
corr_matrix = df_num.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matrice de corrélation des variables numériques")
plt.show()


In [ ]:
# =========================
# 2️⃣ VIF (Variance Inflation Factor)
# =========================

# Ajouter une constante pour le calcul du VIF
X = add_constant(df_num)

vif_data = pd.DataFrame()
vif_data["variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("VIF des variables numériques :")
print(vif_data)

Suppression des données redondantes, risque de multicolinéarité en ML

In [ ]:
colonnes_base = [
    "code_dept", "nom_dept",
    "nb_inscrits", "nb_abstentions", "nb_votants", 
    "nb_blancs", "nb_nuls"
]

df_infos_general_base = df_infos_general[colonnes_base]
df_infos_general_base.head()

In [ ]:
df_infos_general_base.shape

In [ ]:
df_departement = df_infos_general_base.iloc[:, :2]
df_departement.head()

In [ ]:
df_stat_elections = df_infos_general_base.drop(columns=["nom_dept"])
df_stat_elections.head()

### Le test de ki2 pour verifier le lien entre deux variables qualitatif
### Le V2Cramer ou tupro pour mesurer l'intancite du lien

Entre deux varialbles quantitatif il faut faire des boxplot bivarier pour mesurer leurs relations

Test non parametrique entre une variable quantitatif et une variable quantitatif: krukal wallis

Infos sur les departements

Infos résultat des candidats par départements

### Partion

Partion de la base de donnée, tel que la base d'aprentissage = 0,7 et la base de test = 0,3

Tratification pour consever les proportions

In [ ]:
print(df)

In [ ]:
import pandas as pd

# Colonnes globales que l'on veut garder
cols_globales = ['Code du département', 'Libellé du département']

# Colonnes répétitives candidats
df_candidates = df.iloc[:, 17:]  # après % Exp/Vot
cols_per_candidate = 6
num_candidates = df_candidates.shape[1] // cols_per_candidate

dfs_list = []

for i in range(num_candidates):
    start_col = i * cols_per_candidate
    end_col = start_col + cols_per_candidate
    df_cand = df_candidates.iloc[:, start_col:end_col].copy()
    df_cand.columns = ['Sexe', 'Nom', 'Prenom', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    # Ajouter les informations de département
    df_cand = pd.concat([df[cols_globales], df_cand], axis=1)
    
    dfs_list.append(df_cand)

# Combiner tous les candidats
df_simple = pd.concat(dfs_list, ignore_index=True)

# Colonnes à garder
cols_to_keep = ['Code du département', 'Libellé du département', 'Sexe', 'Nom', 'Prenom', 'Voix']

# Nouveau dataframe simplifié
df_candidat_resultat = df_simple[cols_to_keep]

# Afficher toutes les lignes
pd.set_option('display.max_rows', 10)  # ou un nombre supérieur à 100
pd.set_option('display.max_columns', None)  # pour voir toutes les colonnes
pd.set_option('display.width', None)        # pour éviter la coupure à la largeur de l'écran


# Affichage des 5 premières lignes
df_candidat_resultat.head(1000)


In [ ]:
from utils.election_importer import ElectionImporter
from datetime import date
from enums.sexe import SexeEnum
from enums.type_election import TypeElection

# Instanciation
importer = ElectionImporter()

# Import des départements
importer.import_departements(df_departement)

# Créer ou récupérer l’élection
election = importer.get_or_create_election(date(2022, 4, 10), TypeElection.PRESIDENTIELLE)

# Import des stats
importer.import_stats(df_stat_elections, election.id)

# Import des candidats et résultats
importer.import_candidats_resultats(df_candidat_resultat, election.id)


In [ ]:
from utils.election_dataframe import ElectionDataFrame

edf = ElectionDataFrame("./elections/presidentielles-2022-2.xlsx")

In [ ]:
# Liste des candidats
edf.get_candidats()

In [ ]:
# Voix de Mélenchon par département
edf.get_voix_par_departement("LE PEN", "Marine")

In [ ]:
# Stats du département 59
edf.get_stats_departement("59")

In [ ]:
# Stats + voix pour Macron dans le 75
edf.get_stats_departement_candidat("59", "LE PEN", "Marine")

In [ ]:
df_departement = edf.df_departement
print(df_departement.head())

In [ ]:
df_stat_elections = edf.df_stat_elections
print(df_stat_elections.head())

In [ ]:
df_candidat_resultat = edf.df_candidat_resultat
print(df_candidat_resultat.head())